In [1]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
import pandas as pd
from pyspark.sql.types import ArrayType,StringType,DoubleType
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.sql.functions import *

In [2]:
sc=pyspark.SparkContext()

spark= SparkSession \
        .builder \
        .appName("PART 1") \
        .config("spark.some.config.option","15g") \
        .getOrCreate()

In [3]:

# df = spark.read.load("sample-movie-train.csv",format="csv", sep=",",inferSchema='true', header='true')

import pandas as pd
from pyspark.sql import SQLContext
p_df = pd.read_csv("train.csv")
# print(p_df)
sqlCtx = SQLContext(sc)
df = sqlCtx.createDataFrame(p_df)
df.show()
df.printSchema()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [4]:
import pandas as pd
from pyspark.sql import SQLContext
test = pd.read_csv("test.csv")
# print(p_df)
sqlCtx = SQLContext(sc)
test = sqlCtx.createDataFrame(test)
test.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)



In [5]:
df_map = spark.read.load("mapping.csv",format="csv", sep=",",inferSchema='true', header='true')
dd_map= df_map.select("0").collect()[:]

def map_gen(x):
    output_array=[]
    for gen in dd_map:
        if gen[0] in x:
            output_array.append(1)
        else:
            output_array.append(0)
    return output_array
             
mapper = udf(map_gen,ArrayType(StringType()))
genre_mapped = df.withColumn("mapped_genre", mapper("genre"))

In [6]:
num_label = len(dd_map)
result = genre_mapped.select(['movie_id', 'movie_name', 'plot', 'genre', 'mapped_genre']+[expr('mapped_genre[' + str(x) + ']') for x in range(0, num_label)])

new_colnames = ['movie_id', 'movie_name', 'plot', 'genre', 'mapped_genre'] + ['label_' + str(i) for i in range(0, num_label)] 
result = result.toDF(*new_colnames)
result.show()

+--------+--------------------+--------------------+--------------------+--------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|movie_id|          movie_name|                plot|               genre|        mapped_genre|label_0|label_1|label_2|label_3|label_4|label_5|label_6|label_7|label_8|label_9|label_10|label_11|label_12|label_13|label_14|label_15|label_16|label_17|label_18|label_19|
+--------+--------------------+--------------------+--------------------+--------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[1, 0, 0, 0, 0, 1...|      1|      0|      0|      0|      0|      1|      0|      0|      0|      0|       0|       0|       0|    

In [7]:
from pyspark.ml.feature import HashingTF, Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, Word2Vec, PCA
from pyspark.ml import Pipeline
result.show()
test.show()
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
preprocessor = Pipeline(stages=[regexTokenizer, remover])
model = preprocessor.fit(result)
train = model.transform(result)

model = preprocessor.fit(test)
test = model.transform(test)

+--------+--------------------+--------------------+--------------------+--------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|movie_id|          movie_name|                plot|               genre|        mapped_genre|label_0|label_1|label_2|label_3|label_4|label_5|label_6|label_7|label_8|label_9|label_10|label_11|label_12|label_13|label_14|label_15|label_16|label_17|label_18|label_19|
+--------+--------------------+--------------------+--------------------+--------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[1, 0, 0, 0, 0, 1...|      1|      0|      0|      0|      0|      1|      0|      0|      0|      0|       0|       0|       0|    

In [8]:
train.printSchema()
test.printSchema()


root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- mapped_genre: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- label_0: string (nullable = true)
 |-- label_1: string (nullable = true)
 |-- label_2: string (nullable = true)
 |-- label_3: string (nullable = true)
 |-- label_4: string (nullable = true)
 |-- label_5: string (nullable = true)
 |-- label_6: string (nullable = true)
 |-- label_7: string (nullable = true)
 |-- label_8: string (nullable = true)
 |-- label_9: string (nullable = true)
 |-- label_10: string (nullable = true)
 |-- label_11: string (nullable = true)
 |-- label_12: string (nullable = true)
 |-- label_13: string (nullable = true)
 |-- label_14: string (nullable = true)
 |-- label_15: string (nullable = true)
 |-- label_16: string (nullable = true)
 |-- label_17: string (nullable = true)
 |-- label_18: string (nullable = true)
 

In [10]:
from pyspark.ml.feature import HashingTF, IDF

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, Word2Vec, PCA


output = []
for i in range(num_label):
  label = "label_"+str(i)
  print(label)
  cur_train = train.select("movie_id", "movie_name", "filtered", "plot", label)
  cur_train = cur_train.withColumn(label, train[label].cast('float')).select("movie_id", "movie_name", "plot", "filtered", label)
  cv = CountVectorizer(inputCol="filtered", outputCol="count_features", minDF=5)

  lr = LogisticRegression(maxIter=500, regParam=0.001, featuresCol = 'count_features', labelCol=label)
  pipeline = Pipeline(stages=[cv, lr])
  # Fit the pipeline to training documents.
  model = pipeline.fit(cur_train)

  prediction = model.transform(test)
  # prediction.show(5)
  res = prediction.select('movie_id', 'prediction').collect()
  output.append(res)

  

label_0
label_1
label_2
label_3
label_4
label_5
label_6
label_7
label_8
label_9
label_10
label_11
label_12
label_13
label_14
label_15
label_16
label_17
label_18
label_19


In [11]:
print(len(output))
print(len(output[0]))

from collections import defaultdict
op_dic = defaultdict(list)
for row in output:
  for i in row:
    op_dic[i["movie_id"]].append(i['prediction'])

print(len(op_dic))
print(len(op_dic[21465561]))

import csv
with open('sample.csv', 'w') as f:
    f.write("%s,%s\n"%('movie_id', 'predictions'))
    for key in op_dic.keys():
        f.write("%s,%s\n"%(key, ' '.join(str(e)[0] for e in op_dic[key])))

20
7777
7777
20
